In [8]:
import os

import torch
import torch.nn as nn
from torch.utils.data import dataset
from torchvision import transforms


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!mkdir -p /content/drive/MyDrive/data/satet

In [ ]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip -O /content/drive/MyDrive/data/testSet.zip

In [ ]:
!unzip -q /content/drive/MyDrive/data/testSet.zip -d /content/drive/MyDrive/data/

In [ ]:
!git clone https://github.com/1-1is0/deep-projects

In [ ]:
!cp /content/deep-projects/q2/*.py .

In [2]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


/home/amir/uni/deep-vision/venv/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
if os.environ.get("USERNAME") == 'amir':
    base_path = "."
else:
    base_path = "/content/drive/MyDrive"
data_path = f"{base_path}/data/tiny-imagenet-200/train/*"
state_path = f"{base_path}/data/state"


In [12]:
from dataloader import load_data, ImageDataset, Rescale, ToTensor, GetAlexConv
train_df, test_df = load_data(data_path)
train_dataset = ImageDataset(
    train_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    )
)
test_dataset = ImageDataset(
    test_df,
    transform=transforms.Compose(
        [
            Rescale((227, 227)),
            ToTensor(),
        ]
    )
)


trainloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, num_workers=0
)
testloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=64, shuffle=True, num_workers=0
)

print("train_dataset:", len(train_dataset))
print("test_dataset", len(test_dataset))


train_dataset: 100
test_dataset 20


In [13]:
from model import AlexConv2
net = AlexConv2().to(device)
net.initialize_weights()
print(net)

AlexConv2(
  (alexnet): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=9216, out_features=4096, bias=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=4096, out_features=4096, bias=True)
      (5): ReLU(inplace=True)
      (6): Linear(in_features=4096, out_features=1000, bias=True)
    )
  )
  (conv1): Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(256, 256, kerne

In [14]:
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, StepLR

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

scheduler = StepLR(optimizer, step_size=5, gamma=0.3)


In [15]:

from model import train_model
dataloader = {
    "train": trainloader,
    "val": testloader,
}
dataset_size = {
    "train": len(train_dataset),
    "val": len(test_dataset),
}

res = train_model(data_loader=dataloader, model=net, criterion=criterion, optimizer=optimizer, scheduler=scheduler, device=device, epochs=20, dataset_size=dataset_size, path=state_path)


epoch 0 14.703397631645203
epoch 0 5.039158463478088
epoch 1 14.662000596523285
epoch 1 5.0068652629852295


KeyboardInterrupt: 

In [11]:
i = train_dataset[0]['image']
ii = i.unsqueeze(0)
print(ii.shape)
ans = net(ii.to(device))
print(ans.shape)

torch.Size([1, 3, 227, 227])
torch.Size([1, 3, 416, 416])


In [25]:
from torchvision import models

alexnet = models.alexnet(pretrained=True)
for params in alexnet.parameters():
    params.requires_grad = False
alexnet.classifier  = alexnet.classifier[:-4]

In [26]:
im = train_dataset[0]['image']
im = im.unsqueeze(0)
print('im shape', im.shape)
x = alexnet(im.to(device))
print('x shape', x.shape)

im shape torch.Size([1, 3, 227, 227])
x shape torch.Size([1, 4096])


In [25]:
conv1 = nn.Conv2d(192, 256, kernel_size=3, stride=1, padding=1)
conv2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
conv3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
upconv1 = nn.ConvTranspose2d(256, 256, kernel_size=5, stride=2, padding=2, output_padding=1)
upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1)
upconv3 = nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2, padding=2, output_padding=1)
upconv4 = nn.ConvTranspose2d(64, 32, kernel_size=5, stride=2, padding=2, output_padding=1)
upconv5 = nn.ConvTranspose2d(32, 3, kernel_size=5, stride=2, padding=2, output_padding=1)

out1 = conv1(x)
print('out1 shape', out1.shape)
out2 = conv2(out1)
print('out2 shape', out2.shape)
out3 = conv3(out2)
print('out3 shape', out3.shape)
out4 = upconv1(out3)
print('out4 shape', out4.shape)
out5 = upconv2(out4)
print('out5 shape', out5.shape)
out6 = upconv3(out5)
print('out6 shape', out6.shape)
out7 = upconv4(out6)
print('out7 shape', out7.shape)
out8 = upconv5(out7)
print('out8 shape', out8.shape)




out1 shape torch.Size([1, 256, 13, 13])
out2 shape torch.Size([1, 256, 13, 13])
out3 shape torch.Size([1, 256, 13, 13])
out4 shape torch.Size([1, 256, 26, 26])
out5 shape torch.Size([1, 128, 52, 52])
out6 shape torch.Size([1, 64, 104, 104])
out7 shape torch.Size([1, 32, 208, 208])
out8 shape torch.Size([1, 3, 416, 416])


In [27]:

fc1 = nn.Linear(4096, 4096)
fc2 = nn.Linear(4096, 4096)
fc3 = nn.Linear(4096, 4096)
upconv1 = nn.ConvTranspose2d(
    256, 256, kernel_size=5, stride=2, padding=2, output_padding=1
)
upconv2 = nn.ConvTranspose2d(
    256, 128, kernel_size=5, stride=2, padding=2, output_padding=1
)
upconv3 = nn.ConvTranspose2d(
    128, 64, kernel_size=5, stride=2, padding=2, output_padding=1
)
upconv4 = nn.ConvTranspose2d(
    64, 32, kernel_size=5, stride=2, padding=2, output_padding=1
)
upconv5 = nn.ConvTranspose2d(
    32, 3, kernel_size=5, stride=2, padding=2, output_padding=1
)

out1 = fc1(x)
print('out1 shape', out1.shape)
out2 = fc2(out1)
print('out2 shape', out2.shape)
out3 = fc3(out2)
print('out3 shape', out3.shape)
out33 = out3.view(x.size(0), 256, 4, 4)
print("out33 shape", out33.shape)
out4 = upconv1(out33)
print('out4 shape', out4.shape)
out5 = upconv2(out4)
print('out5 shape', out5.shape)
out6 = upconv3(out5)
print('out6 shape', out6.shape)
out7 = upconv4(out6)
print('out7 shape', out7.shape)
out8 = upconv5(out7)



out1 shape torch.Size([1, 4096])
out2 shape torch.Size([1, 4096])
out3 shape torch.Size([1, 4096])
out33 shape torch.Size([1, 256, 4, 4])
out4 shape torch.Size([1, 256, 8, 8])
out5 shape torch.Size([1, 128, 16, 16])
out6 shape torch.Size([1, 64, 32, 32])
out7 shape torch.Size([1, 32, 64, 64])


In [29]:
import torch.nn.functional as F

In [30]:
F.interpolate(out8, 277).shape

torch.Size([1, 3, 277, 277])